In [ ]:
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import geoviews as gv
import rioxarray as rx
import os
import panel as pn
from holoviews import opts
hv.extension('bokeh', width=90)
pn.extension()
import xrspatial as xrs
from decimal import Decimal

In [ ]:
places={}
places['St.Thomas']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif"
places['St.Croix']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n18w065/USGS_1_n18w065.tif"
places['Cornell']='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w076/USGS_1_n42w076.tif'

In [ ]:
maps = ['inferno','blues','kb','kgy','viridis']

Create Global Widgets

In [ ]:
locationwidget = pn.widgets.Select(name='Select Location', options=places)
cmapwidget = pn.widgets.Select(name='Select Color Map', options=maps)

In [ ]:
data = 0
maxheight = 0
minheight = 0

Create the display for tabs

In [ ]:
def options(loc,colormap):
    global data
    global maxheight
    global minheight
    data = xr.open_rasterio(loc)
    data = data.where(data>0)[0,:,:]
    maxheight = round(float(data.where(data>0).max()))
    minheight = round(float(data.where(data>0).min()))
    heightrangeslider.start = minheight
    heightrangeslider.end = maxheight
    heightrangeslider.value=(minheight,maxheight)

In [ ]:
def shaded(loc):
    global data
    shade = xrs.hillshade(data)
    return shade.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget, alpha = 20)

In [ ]:
def heightrange(border,loc):
    global data
    manipulate = data.where(np.logical_and(data>border[0],data<border[1]))
    shade = xrs.hillshade(data)
    return shade.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
heightrangeslider=pn.widgets.RangeSlider(step=1,start = minheight, end = maxheight, value=(minheight,maxheight))

In [ ]:
option = pn.interact(options, loc = locationwidget, colormap = cmapwidget)
terrainshade = pn.interact(shaded, loc = locationwidget)
heightregion = pn.interact(heightrange, border = heightrangeslider, loc = locationwidget)

In [ ]:
dashboard = pn.Tabs(('Options',option),("Shaded View",terrainshade),('Height Quantile',heightregion))

In [ ]:
dashboard.servable()

In [ ]:
print(locationwidget.value)